In [28]:
# !nvidia-smi

In [1]:
!pwd

/disk_ext/nlp


In [29]:
# !pip install tensorflow==1.5.0

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os
from tqdm import tqdm

In [2]:
# from tf.keras.models import Model  # This does not work!
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
tf.__version__, tf.keras.__version__

('2.1.0', '2.2.4-tf')

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('./data_text/pmindia.v1.hi-en.tsv', sep = '\t', header =0 )

In [6]:
df.shape

(56830, 2)

In [7]:
df.head()

,An advance is placed with the Medical Superintendents of such hospitals who then provide assistance on a case to case basis.,"अग्रिम धन राशि इन अस्पतालों को चिकित्सा निरीक्षकों को दी जाएगी, जो हर मामले को देखते हुए सहायता प्रदान करेंगे।"
0,Since the DoHFW provides funds to the hospital...,चूंकि स्वास्थ्य एवं परिवार कल्याण विभाग अस्पता...
1,"RAN functions can, therefore, be vested in DoHFW.",इस तरह आरएएन का कामकाज स्वास्थ्य एवं परिवार कल...
2,Managing Committee of RAN Society will meet to...,"आरएएन, सोसायटी की प्रबंध समिति सोसायटी पंजीकरण..."
3,"In addition to this, Health Minister’s Cancer ...",इसके अलावा स्वास्थ्य मंत्री के कैंसर रोगी निधि...
4,The timeline required for this is one year.,इसके लिए एक वर्ष का समय रखा गया है।


In [8]:
df.loc[0,:]

An advance is placed with the Medical Superintendents of such hospitals who then provide assistance on a case to case basis.    Since the DoHFW provides funds to the hospital...
अग्रिम धन राशि इन अस्पतालों को चिकित्सा निरीक्षकों को दी जाएगी, जो हर मामले को देखते हुए सहायता प्रदान करेंगे।                  चूंकि स्वास्थ्य एवं परिवार कल्याण विभाग अस्पता...
Name: 0, dtype: object

In [9]:
data_src = df[df.columns[0]]


In [10]:
mark_start = 'ssss '
mark_end = ' eeee'
data_dest = []
for line in tqdm(df[df.columns[1]]):
    output_sentence =  mark_start +  line + mark_end
    data_dest.append(output_sentence)


100%|██████████| 56830/56830 [00:00<00:00, 1152572.16it/s]


### Tokenizer


In [11]:
num_words = 40000

In [12]:
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Reverse the tokens.
            tokens = np.flip(tokens, axis=1)

            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        if padding:
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

In [13]:
%%time
tokenizer_src = TokenizerWrap(texts=data_src,
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

CPU times: user 2.46 s, sys: 29.1 ms, total: 2.48 s
Wall time: 2.48 s


In [14]:
%%time
tokenizer_dest = TokenizerWrap(texts=data_dest,
                               padding='post',
                               reverse=False,
                               num_words=num_words)

CPU times: user 3.58 s, sys: 25.7 ms, total: 3.61 s
Wall time: 3.61 s


In [15]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(56830, 38)
(56830, 43)


In [16]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

2

In [17]:
tokenizer_src.tokens_to_string(tokens_src[2]), tokenizer_dest.tokens_to_string(tokens_dest[2])

('sra 1860 act registration societies of provisions per as ab body autonomous the dissolve to meet will society ran of committee managing',
 'ssss आरएएन सोसायटी की प्रबंध समिति सोसायटी पंजीकरण अधिनियम 1860 के प्रावधानों के तहत स्वायत्तशासी निकायों को रद्द करने के लिए बैठक करेगा। eeee')

In [18]:
tokens_dest[2].shape

(43,)

In [19]:
encoder_input_data = tokens_src
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]
decoder_output_data.shape, decoder_input_data.shape

((56830, 42), (56830, 42))

In [20]:
encoder_input = Input(shape=(None, ), name='encoder_input')

embedding_size = 128
encoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='encoder_embedding')
state_size = 512
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)

def connect_encoder():
    # Start the neural network with its input-layer.
    net = encoder_input
    
    # Connect the embedding-layer.
    net = encoder_embedding(net)

    # Connect all the GRU-layers.
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    # This is the output of the encoder.
    encoder_output = net
    
    return encoder_output


In [21]:
encoder_output = connect_encoder()

### Decoder

In [22]:

decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')
decoder_input = Input(shape=(None, ), name='decoder_input')

decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)

decoder_dense = Dense(num_words,
                      activation='softmax',
                      name='decoder_output')

def connect_decoder(initial_state):
    # Start the decoder-network with its input-layer.
    net = decoder_input

    # Connect the embedding-layer.
    net = decoder_embedding(net)
    
    # Connect all the GRU-layers.
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    # Connect the final dense layer that converts to
    # one-hot encoded arrays.
    decoder_output = decoder_dense(net)
    
    return decoder_output


In [23]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

In [24]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])


In [25]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

In [28]:
model_train.compile(optimizer=RMSprop(lr=1e-3),
                    loss='sparse_categorical_crossentropy')

path_checkpoint = './checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=3, verbose=1)

callback_tensorboard = TensorBoard(log_dir='./logs/',
                                   histogram_freq=0,
                                   write_graph=False)

callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard]

In [27]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)
    

Error trying to load checkpoint.
Shapes (40000, 128) and (20000, 128) are incompatible


In [29]:
    
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

y_data = \
{
    'decoder_output': decoder_output_data
}

validation_split = 10000 / len(encoder_input_data)

In [ ]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=384,
                epochs=10,
                validation_split=validation_split,
                callbacks=callbacks)

Train on 46830 samples, validate on 10000 samples
Epoch 1/10
46464/46830 [============================>.] - ETA: 13s - loss: 4.0557
Epoch 00001: val_loss improved from inf to 3.52699, saving model to ./checkpoint.keras
46830/46830 [==============================] - 1909s 41ms/sample - loss: 4.0518 - val_loss: 3.5270
Epoch 2/10
44160/46830 [===========================>..] - ETA: 1:47 - loss: 3.3143
Epoch 00002: val_loss improved from 3.52699 to 3.22249, saving model to ./checkpoint.keras
46830/46830 [==============================] - 2001s 43ms/sample - loss: 3.3041 - val_loss: 3.2225
Epoch 3/10
46464/46830 [============================>.] - ETA: 13s - loss: 3.0783
Epoch 00003: val_loss improved from 3.22249 to 2.99853, saving model to ./checkpoint.keras
46830/46830 [==============================] - 1938s 41ms/sample - loss: 3.0763 - val_loss: 2.9985
Epoch 4/10
46464/46830 [============================>.] - ETA: 14s - loss: 2.8351
Epoch 00004: val_loss improved from 2.99853 to 2.86712,

In [162]:
def translate(input_text, true_output_text=None):
    """Translate a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding=True)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)

    # Max number of tokens / words in the output sequence.
    max_tokens = tokenizer_dest.max_tokens

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict(), but it would make the code
        # much more complicated.

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()

    # Print the translated output-text.
    print("Translated text:")
    print(output_text)
    print()

    # Optionally print the true translated text.
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

In [164]:
data_dest[3]

'ssss इसके अलावा स्वास्थ्य मंत्री के कैंसर रोगी निधि को भी विभाग को स्थानांतरित कर दिया जाएगा। eeee'

In [166]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

3

In [167]:
idx = 3
translate(input_text=data_src[idx],
          true_output_text=data_dest[idx])

Input text:
In addition to this, Health Minister’s Cancer Patient Fund (HMCPF) shall also be transferred to the Department.

Translated text:
 इस परियोजना के तहत इस प्रकार की अवधि में भी कमी आएगी। eeee

True output text:
ssss इसके अलावा स्वास्थ्य मंत्री के कैंसर रोगी निधि को भी विभाग को स्थानांतरित कर दिया जाएगा। eeee



In [169]:
idx = 22
translate(input_text=data_src[idx],
          true_output_text=data_dest[idx])

Input text:
He said the Speaker’s name itself suggests that everyone will feel she is a friend.

Translated text:
 उन्‍होंने कहा कि यह एक बार फिर है जो भारत के लिए भी प्रेरणा नहीं है। eeee

True output text:
ssss उन्‍होंने कहा कि अध्‍यक्ष का नाम ही सभी को उनका मित्र होने का एहसास कराता है। eeee

